# ICN Programming Course

<p align="center">
    <img width="500" alt="image" src="https://github.com/Lenakeiz/ICN_Programming_Course/blob/main/Images/cog_neuro_logo_blue_png_0.png?raw=true">
</p>

---

# **WEEK 5** - Handling files, exception

## Working with files

After conducting your experiments results are typically stored in files. For example:

 - A text file (.txt) may contain timestamps of neuronal spikes, each recorded on a new line.

 - A CSV file (.csv) may contain structured data such as subject IDs, ages, conditions, and behavioural accuracy across trials.

 - Specific formats that depends on the information you have recorded (fMRI, EEG, etc)

Being able to read data from files is then the starting step from any analysis pipeline. 

Similarly when running an experiment you will need to write back data into files for saving your analysis outputs—so that you or other researchers can check, share, and build on your work.

### Opening a file

The basic way to open a file in Python is using the `open()` function.

- The **first argument** is the file name (e.g. `"example.txt"`).  
  - This can be a **relative path** (just the file name, if the file is in the same folder as your script).  
  - Or it can be a **full (absolute) path**, for example:  
    - On Windows:  
      ```python
      file = open("C:\\Users\\Andrea\\Documents\\data\\example.txt", "r")
      ```
      *(note the double backslashes)*  
    - On macOS/Linux:  
      ```python
      file = open("/Users/andrea/Documents/data/example.txt", "r")
      ```

- The **second argument** is the *mode*, which tells Python what you want to do with the file.  

#### Common File Modes

| Mode | Meaning | Notes |
|------|---------|-------|
| `'r'` | Read | Default mode. Opens the file for reading. Fails if the file doesn’t exist. |
| `'w'` | Write | Creates a new empty file or overwrites an existing one. |
| `'a'` | Append | Opens a file for writing, but adds content to the end (does not erase existing data). |
| `'x'` | Create | Creates a new file. Fails if the file already exists. |
| `'r+'` | Read and Write | Opens an existing file for both reading and writing. |

In [1]:
file = open("data\example.txt", "r")  # "r" means read mode
content = file.read()
print(content)
file.close()  # Always close the file after use

10 15 20 35 40
55 60 75 90 120
130 145 160 170 185
200 210 225 240 250
270 280 300 310 325
340 355 370 385 400
420 435 450 465 480
500 520 540 560 580
600 620 640 660 680
700 720 740 760 780


### Using a Context Manager (with)

When working with files, it’s important to close them once you are done.

Closing a file frees up system resources and ensures that any changes are properly saved.

If you forget to close a file, you may run into problems such as:  

- **Memory leaks**: the program holds resources longer than needed, although this will not happen when opening files into jupyter notebooks     
- **File locks**: some operating systems prevent other programs from accessing a file while it is open 

To avoid these issues, Python provides a convenient structure called a **context manager**, used with the `with` statement.  

When you open a file inside a `with` block:  
- The file is automatically **closed** as soon as the block finishes, even if an error occurs inside the block.  
- This makes your code shorter, cleaner, and less error-prone compared to manually calling `file.close()`.  


In [2]:
with open("data\example.txt", "r") as file:
    content = file.read()
    print(content)

10 15 20 35 40
55 60 75 90 120
130 145 160 170 185
200 210 225 240 250
270 280 300 310 325
340 355 370 385 400
420 435 450 465 480
500 520 540 560 580
600 620 640 660 680
700 720 740 760 780


### Creating the path

When working with files, it’s important to build paths in a way that works on **any operating system**.  
As mentioned, Windows uses backslashes (`\`) while macOS/Linux use forward slashes (`/`).

If you type the path manually, you might get an error depending on which operating system you are running the notebook.  

To stay operating system independent, use the `os` library:

In [ ]:
import os 

path = os.path.join("data", "example.txt")

with open(path, "r") as file:    
    content = file.read()
    print(content)

10 15 20 35 40
55 60 75 90 120
130 145 160 170 185
200 210 225 240 250
270 280 300 310 325
340 355 370 385 400
420 435 450 465 480
500 520 540 560 580
600 620 640 660 680
700 720 740 760 780


### Reading line by line

Sometimes you would have to read a stored data line by line, for preprocessing reasons.
Reading line by line lets you parse and clean each row safely before storing it.

You can do this with a for loop on your file object.

In [ ]:
import os 
import numpy as np

file_path = os.path.join("data", "example.txt")

# Start with empty arrays
col1 = np.array([], dtype=int)
col2 = np.array([], dtype=int)
col3 = np.array([], dtype=int)
col4 = np.array([], dtype=int)
col5 = np.array([], dtype=int)

with open(file_path, "r") as f:
    for line in f:
        # removing leading/trailing whitespace and splitting by spaces
        # split() returns a list of strings
        parts = line.strip().split()
        if not parts:
            continue

        a, b, c, d, e = (int(x) for x in parts)

        # Append each value to the corresponding NumPy array
        col1 = np.append(col1, a)
        col2 = np.append(col2, b)
        col3 = np.append(col3, c)
        col4 = np.append(col4, d)
        col5 = np.append(col5, e)

# Quick check
print(col1[:5])  
print(col2[:5])
print("Shapes:", col1.shape, col2.shape, col3.shape, col4.shape, col5.shape)

[ 10  55 130 200 270]
[ 15  60 145 210 280]
Shapes: (10,) (10,) (10,) (10,) (10,)


## Writing to a file

in real experiments you don’t just analyse data once and forget about it.

You often need to:
- **Save results** so you can reuse them later without re-running the entire analysis.  
- **Share data** with collaborators, who sometimes use different programming languages to yours (MATLAB, R, etc).  
- **Document your work**: a file output is a permanent record of what your code produced at that time.  
- **Feed into pipelines**: later steps in your research may take your processed data as input. 

<p align="center">
    <img width="500" alt="image" src="https://github.com/Lenakeiz/ICN_Programming_Course/blob/main/week_5/data_files_comics.png?raw=true">
</p>